In [2]:
import numpy as np
from numpy import array
from scipy.cluster.vq import kmeans2
from sklearn.preprocessing import MinMaxScaler
import pandas as pd
import plotly.graph_objs as go
from sklearn.linear_model import LogisticRegression

n = 100

info = {
    's1': np.random.randint(0, 100, n),
    's2': np.random.randint(0, 100, n),
    's3': np.random.randint(0, 100, n)
}

data = pd.DataFrame(info, columns=['s1', 's2', 's3'])
features = array(list(data.values))
min_max_scaler = MinMaxScaler()
whitened = min_max_scaler.fit_transform(features)
centroid, label = kmeans2(whitened, [
    [0, 0, 0],
    [1, 1, 1]], minit='matrix')
counts = np.bincount(label)
df = pd.DataFrame(data=whitened, columns=["s1", "s2", 's3'])
df['label'] = label

plot = go.Figure(data=[go.Scatter3d(x=[data.s1], y=[data.s2], z=[data.s3],
                                    mode='markers', marker=dict(color=['black']))])

for C in list(df.label.unique()):
    plot.add_trace(go.Scatter3d(x=df[df.label == C]['s1'],
                                y=df[df.label == C]['s2'],
                                z=df[df.label == C]['s3'],
                                mode='markers', marker_size=8, marker_line_width=1,
                                name='Cluster ' + str(C)))

plot.show()

X = df[['s1', 's2', 's3']]
y = df['label']
model = LogisticRegression(solver='liblinear', random_state=0)
model.fit(X, y)

x_axis, y_axis = np.meshgrid(range(2), range(2))
z_axis = (-model.intercept_[0] - model.coef_[0][0] * x_axis - model.coef_[0][1] * y_axis) / model.coef_[0][2]

plot = go.Figure()

for C in list(df.label.unique()):
    plot.add_trace(go.Scatter3d(x=df[df.label == C]['s1'],
                                y=df[df.label == C]['s2'],
                                z=df[df.label == C]['s3'],
                                mode='markers', marker_size=8, marker_line_width=1,
                                name='Cluster ' + str(C)))

plot.add_trace(go.Surface(x=x_axis, y=y_axis, z=z_axis))
plot.show()
